# Question 1 - Extracting Tesla Stock Data Using yfinance

In [ ]:

import yfinance as yf
import pandas as pd

# Завантажуємо дані про Tesla
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()


# Question 2 - Extracting Tesla Revenue Data Using Webscraping

In [ ]:

from bs4 import BeautifulSoup
import requests

url = "https://finance.yahoo.com/quote/TSLA/financials?p=TSLA"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

# Перевіряємо заголовок сторінки
print(soup.title)


# Question 3 - Extracting GameStop Stock Data Using yfinance

In [ ]:

gamestop = yf.Ticker("GME")
gme_data = gamestop.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()


# Question 4 - Extracting GameStop Revenue Data Using Webscraping

In [ ]:

url = "https://finance.yahoo.com/quote/GME/financials?p=GME"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

# Перевіряємо заголовок сторінки
print(soup.title)


# Question 5 - Tesla Stock and Revenue Dashboard

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(tesla_data["Date"], tesla_data["Close"], label="Tesla Stock Price", color="blue")
plt.title("Tesla Stock Price Over Time")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()


# Question 6 - GameStop Stock and Revenue Dashboard

In [ ]:

plt.figure(figsize=(10,6))
plt.plot(gme_data["Date"], gme_data["Close"], label="GameStop Stock Price", color="orange")
plt.title("GameStop Stock Price Over Time")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()


# Question 7 - Sharing your Assignment Notebook

Upload your completed notebook to GitHub and include the link in your submission.